In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit

In [ ]:
import requests
import io
import os

def _csv_from_path_or_url(path, url):
    if os.path.isfile(path):
        return path
    else:
        r = requests.get(url)
        r.raise_for_status()
        # After fetching from the URL, cache at the path
        contents = r.text
        with open(path, "w") as f:
            f.write(contents)
        return path

In [ ]:
import inspect
def format_fit_vars(ansatz, fit_, cov):
    arg_names = inspect.getargs(ansatz.__code__).args[1:]
    stddevs = np.sqrt(cov.diagonal())
    values = zip(arg_names, fit_, stddevs)
    return " ".join(f"{arg}={val:0.2f}±{std:0.1f}" for (arg, val, std) in values)

In [ ]:
csv = _csv_from_path_or_url(
    path="south-korea-active-cases.csv",
    url="https://qbin.io/bs-mixed-gv7a/raw",
)
df = pd.read_csv(csv)
(df.info(), df.head(), df.plot(y='active_cases'))

In [ ]:
def gaussian(x, a, b, m, y0):
    return y0 + a * np.exp(-(x - m)**2 / b)
gaussian.initial_guess = [1e4, 10, 20, 0]

def gaussian_dusty(x, y0, A, w, xc):
    four_ln2 = 4 * np.log(2)
    norm_denom = w * np.sqrt(np.pi / four_ln2)
    return y0 + A / norm_denom * np.exp(-four_ln2*(x - xc)**2/w**2)
gaussian_dusty.initial_guess_guess = [0, 2e5, 25, 2e6]

In [ ]:
ansatz = gaussian_dusty

data_kept = df[df['days_since_feb_17'] < 25]
(fit_, cov) = curve_fit(
    ansatz,
    xdata=data_kept['days_since_feb_17'],
    ydata=data_kept['active_cases'],
    p0=ansatz.initial_guess if hasattr(ansatz, 'initial_guess') else None,
)
print(format_fit_vars(ansatz, fit_, cov))
df['gaussian_estimate'] = [ansatz(x, *fit_) for x in df.index]
ax = df.plot(y=['active_cases', 'gaussian_estimate'])